# Geoapify Places 5 Cities

In [1]:
# Dependencies
from census import Census
import hvplot.pandas
import time
import requests
import json
import pandas as pd
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import the API key
from config_key import geoapify_key
from config_key import census_key

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [2]:
# Create an instance of the Census library
# 
c = Census(
    census_key, 
    year=2021
)

# Run Census Search to retrieve data on all zip codes (2021 ACS5 Census)
census_data = c.acs5.get(
    (
        "B01003_001E",
        "B17001_002E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column renaming
census_df = census_df.rename(
    columns = {
        "B01003_001E": "Population",
        "B17001_002E": "Poverty Count",
        "zip code tabulation area": "Zipcode",
        #rural vs urban,
        #latitude and latitude
    }
)

In [3]:
census_df

,Population,Poverty Count,Zipcode
0,17126.0,11302.0,00601
1,37895.0,17121.0,00602
2,49136.0,23617.0,00603
3,5751.0,3139.0,00606
4,26153.0,11640.0,00610
...,...,...,...
33769,13.0,0.0,99923
33770,917.0,182.0,99925
33771,1445.0,252.0,99926
33772,11.0,0.0,99927


In [4]:
#batch API call
#url = "https://api.geoapify.com/v1/batch?apiKey=geoapify_key
#headers = {"Content-Type": "application/json"}
#data = '{"api":"/v2/places","params":{},"inputs":[{"id":"optional-input-id1","params":{}}]}'
      
#try:
#    resp = requests.post(url, headers=headers, data=data)
#    print(resp.json())
#except requests.exceptions.HTTPError as e:
#    print (e.response.text)

In [5]:
cities = [#"Bronx, NY",
          "Napa, CA", "Victorville, CA","San Francisco, CA"]

places = [#"512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c4692#5ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]
rows = []
responses = []

In [6]:
for place in places:
    filters = f"place:{place}"
    categories = "commercial.supermarket"
    #radius = 16100 # 10 miles
    #bias

    params = {
        "categories":categories,
        "apiKey":geoapify_key,
        "filter":filters #,
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url,params=params)

    # convert response to json
    supermarket_data = response.json()

    # print the response url, avoid doing for public github repos in order to avoid exposing key
    print(response.url)
    responses.append(supermarket_data)

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512f63af1c4692%235ec0597f784c9308264340f00101f90177b4010000000000c00208
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208
https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208


In [7]:
# r = []

for elephant in responses:
    # Print the json (pretty printed)
    # print(json.dumps(response, indent=4, sort_keys=True))

    # Extracting 'features' and 'properties' from JSON
    features = elephant.get("features", [])

    # Extracting results from JSON 
    # results = supermarket_data.get("results", []) # Create a list to store rows 

    for feature in features: 
        properties = feature.get("properties",{})
        # Extract relevant information
        name = properties.get("name") 
        place_id = properties.get("place_id")
        lat = properties.get("lat") 
        lon = properties.get("lon") # Append data to the list
        postcode = properties.get("postcode")
        radius = properties.get("radius")
        city = properties.get("city")
        county = properties.get("county")
        rows.append({"Name": name, "City": city, "Zipcode": postcode, "Latitude": lat, "Longitude": lon,"Place Id": place_id, "county":county}) 
    print(len(rows))
#    r.append(rows)

0
10
30


In [8]:
#r
#len(r[3])

In [9]:
# Create a DataFrame 
supermarket_df = pd.DataFrame(rows) # Display the DataFrame 

supermarket_df["City"].value_counts()

City
San Francisco    20
Victorville      10
Name: count, dtype: int64

In [10]:
supermarket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       30 non-null     object 
 1   City       30 non-null     object 
 2   Zipcode    30 non-null     object 
 3   Latitude   30 non-null     float64
 4   Longitude  30 non-null     float64
 5   Place Id   30 non-null     object 
 6   county     10 non-null     object 
dtypes: float64(2), object(5)
memory usage: 1.8+ KB


In [11]:
supermarket_df.columns

Index(['Name', 'City', 'Zipcode', 'Latitude', 'Longitude', 'Place Id',
       'county'],
      dtype='object')

In [12]:
supermarket_df

,Name,City,Zipcode,Latitude,Longitude,Place Id,county
0,Walmart Supercenter,Victorville,92392,34.509436,-117.396774,51a1af947d65595dc059a8d41dc82f414140f00102f901...,San Bernardino County
1,Walmart Supercenter,Victorville,92392,34.465300,-117.354453,51d6295e6fae565dc0593d8733f3893b4140f00102f901...,San Bernardino County
2,WinCo Foods,Victorville,93533,34.523200,-117.329383,518e80be0814555dc0598598d101f8424140f00102f901...,San Bernardino County
3,Vons,Victorville,92392,34.472572,-117.289651,51f14a81bd89525dc059c188f8417d3c4140f00102f901...,San Bernardino County
4,Food 4 Less,Victorville,92392,34.519390,-117.314521,515a7ad91b21545dc05928201b5f7b424140f00103f901...,San Bernardino County
5,Costco,Victorville,92395,34.514864,-117.318645,514983365964545dc0597d416df4e9414140f00102f901...,San Bernardino County
6,Vallarta,Victorville,92392,34.475154,-117.339351,51d0f35dd4b7555dc059901f0604d23c4140f00102f901...,San Bernardino County
7,Stater Bros.,Victorville,92392,34.471840,-117.362971,515e99b7ea3a575dc0597aa8cb40653c4140f00103f901...,San Bernardino County
8,Albertsons,Victorville,92392,34.471805,-117.365750,512540f27368575dc059a9bcd617643c4140f00103f901...,San Bernardino County
9,Stater Bros.,Victorville,93533,34.520182,-117.331506,51611bf16437555dc0599fad345195424140f00103f901...,San Bernardino County


In [13]:
supermarket_df.hvplot.points(
    'Longitude', 
    'Latitude', 
    geo=True, 
    color='red', 
    alpha=0.2,
    xlim=(-123, -121), 
    ylim=(37.6, 38), 
    frame_width = 1600,
    frame_height = 1200,
    tiles='OSM')

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]

In [14]:
supermarket_totals_df = supermarket_df.groupby(['City', 'Zipcode']).size().reset_index(name='Supermarket Count')
supermarket_totals_df

,City,Zipcode,Supermarket Count
0,San Francisco,94103,2
1,San Francisco,94107,2
2,San Francisco,94109,1
3,San Francisco,94110,2
4,San Francisco,94111,1
5,San Francisco,94112,1
6,San Francisco,94114,2
7,San Francisco,94115,1
8,San Francisco,94116,1
9,San Francisco,94118,1


In [15]:
census_df

,Population,Poverty Count,Zipcode
0,17126.0,11302.0,00601
1,37895.0,17121.0,00602
2,49136.0,23617.0,00603
3,5751.0,3139.0,00606
4,26153.0,11640.0,00610
...,...,...,...
33769,13.0,0.0,99923
33770,917.0,182.0,99925
33771,1445.0,252.0,99926
33772,11.0,0.0,99927


In [16]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33774 entries, 0 to 33773
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Population     33774 non-null  float64
 1   Poverty Count  33774 non-null  float64
 2   Zipcode        33774 non-null  object 
dtypes: float64(2), object(1)
memory usage: 791.7+ KB


In [17]:
# Merge datasets on Zipcodes
supermarkets_density_final = pd.merge(
    supermarket_totals_df,
    census_df,
    how = "left",
    on = ["Zipcode", "Zipcode"]
)
supermarkets_density_final

,City,Zipcode,Supermarket Count,Population,Poverty Count
0,San Francisco,94103,2,32430.0,5202.0
1,San Francisco,94107,2,28607.0,2206.0
2,San Francisco,94109,1,56114.0,6850.0
3,San Francisco,94110,2,70859.0,6939.0
4,San Francisco,94111,1,4707.0,858.0
5,San Francisco,94112,1,84477.0,7560.0
6,San Francisco,94114,2,34474.0,1979.0
7,San Francisco,94115,1,34193.0,4195.0
8,San Francisco,94116,1,45510.0,2680.0
9,San Francisco,94118,1,41456.0,2538.0


In [18]:
#census_df[census_df['Zipcode']=='92392']

In [19]:
supermarkets_density_final = supermarkets_density_final.fillna(0)

In [20]:
supermarkets_density_final = supermarkets_density_final[["Population","Poverty Count"]].astype(int)
supermarkets_density_final

,Population,Poverty Count
0,32430,5202
1,28607,2206
2,56114,6850
3,70859,6939
4,4707,858
5,84477,7560
6,34474,1979
7,34193,4195
8,45510,2680
9,41456,2538


In [21]:
# Add a Supermarket Count column (Supermarkets/Population)
#Supermarket_Total = supermarkets_density_final["Supermarket Count"].sum().astype(int)
#Population_Total = supermarkets_density_final["Population"].astype(int).groupby(["Zipcode"])
#Supermarket_Density = Supermarket_Total / Population_Total

# Configure the final DataFrame
#supermarkets_density_final = supermarkets_density_final[
#    [
#        "postcode",
#        "population",
#        "supermarket"
#    ]
#]

# Display DataFrame length and sample data
# print(f"Number of rows in the DataFrame: {len(supermarkets_density_final)}")
#supermarkets_density_final.head()

In [22]:
# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208&limit=100&apiKey=geoapify_key
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208&limit=100&apiKey=geoapify_key

cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]

place_Bronx = "512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205"
place_Napa = "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208"
place_Victorville = "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208"
place_SanFran = "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"
        


# Set the parameters to search for supermarkets 

#latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
#longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
#radius = 16100 # 10 miles
#radius = 1610 # 1 mile
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#filters = f"circle:{longitude},{latitude},{radius}"

filters = f"place:{places[2]}"
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
radius = 16100 # 10 miles
#radius = 1610 # 1 mile
#bias = f"proximity:{longitude},{latitude}"
#limit = 100


# Set up a params dictionary
params = {
    "categories":categories,
    "apiKey":geoapify_key,
#    "limit": limit,
    "filter":filters #,
#    "bias":bias
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Make an API request using the params dictionary
response = requests.get(base_url,params=params)

# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)    

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208


In [23]:
#c.acs5.tables()

In [24]:
# Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208&limit=100&apiKey=YOUR_API_KEY
#https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=place:513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208&limit=100&apiKey=YOUR_API_KEY

cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA"]

places = ["512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205",
          "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208",
          "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208",
          "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"]

place_Bronx = "512ffe11e13a7852c059ed90ac0d5f6c4440f00101f9010ce3930000000000c00205"
place_Napa = "512f63af1c46925ec0597f784c9308264340f00101f90177b4010000000000c00208"
place_Victorville = "512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208"
place_SanFran = "513a596abddf9a5ec05913dd6921b7e34240f00101f90160b5010000000000c00208"
          
#latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
#longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]


# Set the parameters to search for supermarkets 
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
#radius = 16100 # 10 miles
#radius = 1610 # 1 mile
i = 0

# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#filters = f"circle:{longitude},{latitude},{radius}"

filters = f"place:{places[2]}"
categories = "commercial.supermarket"
# This radius is equivalent to 10 miles, which is the definition of food desert for rural cities
radius = 16100 # 10 miles
#radius = 1610 # 1 mile
#bias = f"proximity:{longitude},{latitude}"
#limit = 100


# Set up a params dictionary
params = {
    "categories":categories,
    "apiKey":geoapify_key,
#    "limit": limit,
    "filter":filters #,
#    "bias":bias
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Make an API request using the params dictionaty
response = requests.get(base_url,params=params)

# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)    

https://api.geoapify.com/v2/places?categories=commercial.supermarket&apiKey=bf8e20276a9741e3b5a704ce3a508735&filter=place%3A512461df4ea2525dc0594fd5f3249f444140f00101f901e0b4010000000000c00208


In [25]:
    # convert response to json
    supermarket_data = response.json()

    # Print the json (pretty printed)
    print(json.dumps(supermarket_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -117.39681949157831,
                    34.50927068192078
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Walmart Supercenter",
                "address_line2": "12234 Palmdale Road, Victorville, CA 92392, United States of America",
                "categories": [
                    "building",
                    "building.commercial",
                    "commercial",
                    "commercial.supermarket"
                ],
                "city": "Victorville",
                "country": "United States",
                "country_code": "us",
                "county": "San Bernardino County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
           